### CIFAR-10 Inception Pytorch Pretrained   

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim

In [2]:

# 1. Load CIFAR-10 dataset
transform = transforms.Compose([
    transforms.Resize(299),  # Inception v3 expects 299x299 images
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)) # ImageNet stats
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [9]:

# 2. Load Pretrained Inception v3
model = torchvision.models.inception_v3(pretrained=True, aux_logits=True) # aux_logits=False for simpler training.

# 3. Modify the final fully connected layer
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 10) # 10 output classes

# 4. Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# 5. Move model to GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)


/drv3/hm3/code/python/torch/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/drv3/hm3/code/python/torch/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

In [6]:
def count_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

count_params( model )

25132754

In [ ]:
# 6. Train the network
for epoch in range(2):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        # Access the main output:
        outputs = outputs.logits # or outputs[0]
        labels = labels.long()

        # Debugging checks:
        # print(f"Outputs shape: {outputs.shape}, Labels shape: {labels.shape}")
        if torch.isnan(outputs).any():
            print("NaNs in outputs")
        if outputs.numel() == 0:
            print("outputs is empty")

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

Outputs shape: torch.Size([4, 10]), Labels shape: torch.Size([4])
Outputs shape: torch.Size([4, 10]), Labels shape: torch.Size([4])
Outputs shape: torch.Size([4, 10]), Labels shape: torch.Size([4])
Outputs shape: torch.Size([4, 10]), Labels shape: torch.Size([4])
Outputs shape: torch.Size([4, 10]), Labels shape: torch.Size([4])
Outputs shape: torch.Size([4, 10]), Labels shape: torch.Size([4])
Outputs shape: torch.Size([4, 10]), Labels shape: torch.Size([4])
Outputs shape: torch.Size([4, 10]), Labels shape: torch.Size([4])
Outputs shape: torch.Size([4, 10]), Labels shape: torch.Size([4])
Outputs shape: torch.Size([4, 10]), Labels shape: torch.Size([4])
Outputs shape: torch.Size([4, 10]), Labels shape: torch.Size([4])
Outputs shape: torch.Size([4, 10]), Labels shape: torch.Size([4])
Outputs shape: torch.Size([4, 10]), Labels shape: torch.Size([4])
Outputs shape: torch.Size([4, 10]), Labels shape: torch.Size([4])
Outputs shape: torch.Size([4, 10]), Labels shape: torch.Size([4])
Outputs sh

In [13]:
# 7. Test the network on the test data
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        outputs = outputs.logits # Access the logits tensor
        _, predicted = torch.max(outputs, 1) # No need for .data anymore
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f} %')

Accuracy of the network on the 10000 test images: 90.13 %


In [14]:

# 8. Class-wise accuracy
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        outputs = outputs.logits # Access the logits tensor
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

for i in range(10):
    print(f'Accuracy of {classes[i]:5s} : {100 * class_correct[i] / class_total[i]:.2f} %')

Accuracy of plane : 88.70 %
Accuracy of car   : 93.60 %
Accuracy of bird  : 90.30 %
Accuracy of cat   : 71.80 %
Accuracy of deer  : 93.10 %
Accuracy of dog   : 86.80 %
Accuracy of frog  : 93.70 %
Accuracy of horse : 95.40 %
Accuracy of ship  : 96.20 %
Accuracy of truck : 94.00 %
